In [9]:
import numpy as np
import os
import glob
import joblib
from tqdm import tqdm

from nilearn.input_data import NiftiMasker
import nibabel as nib

import llms_brain_lateralization as lbl
from llms_brain_lateralization import make_dir, standardize

In [10]:
subject_list = np.sort(glob.glob(os.path.join(lbl.fmri_data_resampled, 'sub-EN*')))

fmri_subs_runs = []
for sub_id in tqdm(subject_list):
    sub_id_basename = os.path.basename(sub_id)
    fmri_imgs_sub = sorted(glob.glob(os.path.join(sub_id, '*.nii.gz')))
    fmri_runs = [] # n_runs x n_timesteps x n_voxels
    for fmri_img in fmri_imgs_sub:
        nifti_masker = NiftiMasker(mask_img='mask_lpp_en.nii.gz', detrend=True, standardize=True,
                                   high_pass=1/128, t_r=lbl.t_r)
        fmri_runs.append(nifti_masker.fit_transform(fmri_img))
    fmri_subs_runs.append(fmri_runs)
print(subject_list)


100%|██████████| 49/49 [13:35<00:00, 16.64s/it]

['lpp_en_resampled/sub-EN057' 'lpp_en_resampled/sub-EN058'
 'lpp_en_resampled/sub-EN059' 'lpp_en_resampled/sub-EN061'
 'lpp_en_resampled/sub-EN062' 'lpp_en_resampled/sub-EN063'
 'lpp_en_resampled/sub-EN064' 'lpp_en_resampled/sub-EN065'
 'lpp_en_resampled/sub-EN067' 'lpp_en_resampled/sub-EN068'
 'lpp_en_resampled/sub-EN069' 'lpp_en_resampled/sub-EN070'
 'lpp_en_resampled/sub-EN072' 'lpp_en_resampled/sub-EN073'
 'lpp_en_resampled/sub-EN074' 'lpp_en_resampled/sub-EN075'
 'lpp_en_resampled/sub-EN076' 'lpp_en_resampled/sub-EN077'
 'lpp_en_resampled/sub-EN078' 'lpp_en_resampled/sub-EN079'
 'lpp_en_resampled/sub-EN081' 'lpp_en_resampled/sub-EN082'
 'lpp_en_resampled/sub-EN083' 'lpp_en_resampled/sub-EN084'
 'lpp_en_resampled/sub-EN086' 'lpp_en_resampled/sub-EN087'
 'lpp_en_resampled/sub-EN088' 'lpp_en_resampled/sub-EN089'
 'lpp_en_resampled/sub-EN091' 'lpp_en_resampled/sub-EN092'
 'lpp_en_resampled/sub-EN093' 'lpp_en_resampled/sub-EN094'
 'lpp_en_resampled/sub-EN095' 'lpp_en_resampled/sub-EN09

In [11]:
make_dir(lbl.fmri_data_avg_subject)

In [12]:
for run in range(lbl.n_runs):
    fmri_mean_sub = np.mean([fmri_sub_runs[run] for fmri_sub_runs in fmri_subs_runs], axis=0)
    fmri_mean_sub = standardize(fmri_mean_sub, axis=0)
    filename = os.path.join(lbl.fmri_data_avg_subject, 'average_subject_run-{}.gz'.format(run))
    with open(filename, 'wb') as f:
         joblib.dump(fmri_mean_sub, f, compress=4)

In [13]:
# now compute reliable voxels
from sklearn.linear_model import Ridge
import time

np.random.seed(1234)

n_subjects = len(subject_list)
n_voxels = nifti_masker.n_elements_

alphas = np.logspace(2,7,16)

n_trials = 10

corr_split = []
for i_trial in range(n_trials):
    print('='*80)
    
    idx_random = np.arange(n_subjects)
    np.random.shuffle(idx_random)
    
    idx_group_1 = idx_random[:n_subjects//2]
    idx_group_2 = idx_random[n_subjects//2:]
    
    regressors_runs = [np.mean([fmri_subs_runs[idx_sub][run][10:-10] for idx_sub in idx_group_1], axis=0)
                                   for run in range(lbl.n_runs)]
    fmri_runs = [np.mean([fmri_subs_runs[idx_sub][run][10:-10] for idx_sub in idx_group_2], axis=0)
                                   for run in range(lbl.n_runs)]

    corr_runs = []
    for run_test in range(lbl.n_runs):
        tic = time.time()
        
        runs_train = np.setdiff1d(np.arange(lbl.n_runs), run_test)
        x_train = np.vstack([regressors_runs[run_train] for run_train in runs_train])
        x_test = regressors_runs[run_test]
        y_train = np.vstack([fmri_runs[run_train] for run_train in runs_train])
        y_test = fmri_runs[run_test]
        
        ############ start nested CV 
        #leave another run apart as a validation test
        run_val = runs_train[0]
        runs_train_val = np.setdiff1d(runs_train, run_val)
        x_train_val = np.vstack([regressors_runs[run_train_val] for run_train_val in runs_train_val])
        x_val = regressors_runs[run_val]
        y_train_val = np.vstack([fmri_runs[run_train] for run_train in runs_train_val])
        y_val = fmri_runs[run_val]

        corr_val = []
        for alpha in alphas:
            model = Ridge(alpha=alpha, fit_intercept=False)
            model.fit(x_train_val, y_train_val)
            y_pred = model.predict(x_val)
            corr_tmp = [np.corrcoef(y_val[:,i], y_pred[:,i])[0,1] for i in range(n_voxels)]
            corr_val.append(corr_tmp)    

        idx_best_alpha = np.argmax(np.mean(corr_val, axis=1))
        alpha = alphas[idx_best_alpha]
        ############ end nested CV 
        
        model = Ridge(alpha=alpha, fit_intercept=False)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        corr_tmp = [np.corrcoef(y_test[:,i], y_pred[:,i])[0,1] for i in range(n_voxels)]

        corr_runs.append(corr_tmp)
        
        toc = time.time()
        
        print('run ', run_test, '\t', 'mean = {:.03f}'.format(np.mean(corr_tmp)), '\t',
            'max = {:.03f}'.format(np.max(corr_tmp)), '\t',
            'time elapsed = {:.03f}'.format(toc-tic))

    corr_split.append(np.mean(corr_runs, axis=0))

run  0 	 mean = 0.403 	 max = 0.880 	 time elapsed = 142.696
run  1 	 mean = 0.314 	 max = 0.818 	 time elapsed = 143.490
run  2 	 mean = 0.358 	 max = 0.880 	 time elapsed = 135.828
run  3 	 mean = 0.389 	 max = 0.894 	 time elapsed = 129.069
run  4 	 mean = 0.421 	 max = 0.911 	 time elapsed = 135.817
run  5 	 mean = 0.377 	 max = 0.853 	 time elapsed = 131.598
run  6 	 mean = 0.340 	 max = 0.878 	 time elapsed = 133.902
run  7 	 mean = 0.307 	 max = 0.893 	 time elapsed = 149.508
run  8 	 mean = 0.374 	 max = 0.891 	 time elapsed = 131.399
run  0 	 mean = 0.414 	 max = 0.902 	 time elapsed = 150.064
run  1 	 mean = 0.321 	 max = 0.810 	 time elapsed = 146.368
run  2 	 mean = 0.360 	 max = 0.841 	 time elapsed = 141.750
run  3 	 mean = 0.374 	 max = 0.883 	 time elapsed = 136.499
run  4 	 mean = 0.418 	 max = 0.906 	 time elapsed = 134.589
run  5 	 mean = 0.370 	 max = 0.838 	 time elapsed = 130.021
run  6 	 mean = 0.339 	 max = 0.848 	 time elapsed = 130.289
run  7 	 mean = 0.320 	 

In [24]:
filename = 'corr_group_split_{}trials.gz'.format(n_trials)
with open(filename, 'wb') as f:
     joblib.dump(np.array(corr_split), f, compress=4)